In [1]:
critic_type = 'inpainting'

In [20]:
from PIL import Image
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import PIL
from skimage.measure import compare_psnr, compare_ssim

# Load Data

### Parameters for Data Loading

In [6]:
data_pth = Path('/home/alaa/Dropbox/BPHO Staff/USF')  # path to dataset
lr_dir = f'EM/training/trainsets/lr/'  # input images
hr_dir = f'EM/training/trainsets/hr/'  # target images
model_path = data_pth/f'EM/models/self_sv/{critic_type}'
test_pth = data_pth/f'EM/testing/LR/real-world_SEM'
infer_pth = data_pth/f'EM/testing/self_sv/{critic_type}'

# Modelling

# Visualization

In [7]:
model_list = ['5.4_superres_transfer_3.pkl', '5.27_imagenet_critic_3.pkl']
tag_list = ['transfer_pretrained', 'imagenet']

In [8]:
model_1_name = model_list[0]
model_2_name = model_list[1]

dir1 = f'{model_1_name}/real-world_SEM/'  # pssr images
dir2 = f'{model_2_name}/real-world_SEM/'  # our images

# Modify accordingly
pth1 = infer_pth/dir1
pth2 = infer_pth/dir2

model1_files = sorted(list(pth1.glob('*.tif')))
model2_files = sorted(list(pth2.glob('*.tif')))

print('Processing '+str(len(model1_files))+' files...')

Processing 42 files...


In [16]:
target_dirname = data_pth/f'EM/testing/HR/real-world_SEM/'       # target images
target_files = sorted(list(target_dirname.glob('*.tif')))
print('Processing '+str(len(target_files))+' files...')

Processing 42 files...


In [13]:
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
np.set_printoptions(linewidth=110)

# Set size for visualizations
fig_size = plt.rcParams["figure.figsize"]  # Get current size
print("Current size:", fig_size)
 
# Set figure width to 12 and height to 9
fig_size[0] = 30
fig_size[1] = 24
plt.rcParams["figure.figsize"] = fig_size

Current size: [6.0, 4.0]


In [14]:
def visualize_sample(img_fname1, img_fname2, targ_fname, title1=model_1_name, title2="Target", title3=model_2_name):
    model1_im = PIL.Image.open(img_fname1) # load image to tensor
    targ_im = PIL.Image.open(targ_fname)
    model2_im = PIL.Image.open(img_fname2)
    
    model1_arr = np.asarray(model1_im, np.uint8)
    model2_arr = np.asarray(model2_im, np.uint8)
    targ_arr = np.asarray(targ_im, np.uint8)
    
    model1_psnr = compare_psnr(targ_arr, model1_arr, data_range=255.)
    model1_ssim = compare_ssim(targ_arr, model1_arr, data_range=255.)
    model2_psnr = compare_psnr(targ_arr, model2_arr, data_range=255.)
    model2_ssim = compare_ssim(targ_arr, model2_arr, data_range=255.)
    f, axarr = plt.subplots(1,3) # create visualizations
    axarr[0].imshow(np.squeeze(model1_arr), cmap=plt.cm.gray) # visualize image tensor
    axarr[0].set_title(title1)
    axarr[0].set_xlabel(f"PSNR: {model1_psnr:.2f}, SSIM: {model1_ssim:.2f}")
    axarr[1].imshow(np.squeeze(targ_arr), cmap=plt.cm.gray) # visualize original image file
    axarr[1].set_title(title2)
    axarr[2].imshow(np.squeeze(model2_arr), cmap=plt.cm.gray) # visualize image tensor
    axarr[2].set_title(title3)
    axarr[2].set_xlabel(f"PSNR: {model2_psnr:.2f}, SSIM: {model2_ssim:.2f}")
    plt.show() # show visualization

In [21]:
@interact
def show_sample(sample=33):
    return visualize_sample(model1_files[sample], model2_files[sample], target_files[sample])

interactive(children=(IntSlider(value=33, description='sample', max=99, min=-33), Output()), _dom_classes=('wi…